# Learning

This notebook serves as supporting material for **Lecture 13 - Learning** from the lecture Grundlagen der Künstlichen Intelligenz (IN2062). This notebook is absorbed from [learning.ipynb](https://github.com/aimacode/aima-python/blob/master/learning.ipynb) and uses implementations from [learning.py](https://github.com/aimacode/aima-python/blob/master/learning.py) of the AIMA repository from the book *Artificial Intelligence: A Modern Approach*. Let's start by importing everything from the module:

In [ ]:
from learning import *
from notebook import *
from utils import *

## CONTENTS

* Introduction

* Decision Tree Learner


### Introduction


You can find the code for the dataset here:

In [ ]:
psource(DataSet)

### Class Attributes

* **examples**: Holds the items of the dataset. Each item is a list of values.

* **attrs**: The indexes of the features (by default in the range of [0,f), where *f* is the number of features). For example, `item[i]` returns the feature at index *i* of *item*.

* **attrnames**: An optional list with attribute names. For example, `item[s]`, where *s* is a feature name, returns the feature of name *s* in *item*.

* **target**: The attribute a learning algorithm will try to predict. By default the last attribute.

* **inputs**: This is the list of attributes without the target.

* **values**: A list of lists which holds the set of possible values for the corresponding attribute/feature. If initially `None`, it gets computed (by the function `setproblem`) from the examples.

* **distance**: The distance function used in the learner to calculate the distance between two items. By default `mean_boolean_error`.

* **name**: Name of the dataset.

* **source**: The source of the dataset (url or other). Not used in the code.

* **exclude**: A list of indexes to exclude from `inputs`. The list can include either attribute indexes (attrs) or names (attrnames).

### Class Helper Functions

These functions help modify a `DataSet` object to your needs.

* **sanitize**: Takes as input an example and returns it with non-input (target) attributes replaced by `None`. Useful for testing. Keep in mind that the example given is not itself sanitized, but instead a sanitized copy is returned.

* **classes_to_numbers**: Maps the class names of a dataset to numbers. If the class names are not given, they are computed from the dataset values. Useful for classifiers that return a numerical value instead of a string.

* **remove_examples**: Removes examples containing a given value. Useful for removing examples with missing values, or for removing classes (needed for binary classifiers).


## DECISION TREE LEARNER

### Overview

#### Decision Trees
A decision tree is a flowchart that uses a tree of decisions and their possible consequences for classification. At each non-leaf node of the tree an attribute of the input is tested, based on which corresponding branch leading to a child-node is selected. At the leaf node the input is classified based on the class label of this leaf node. The paths from root to leaves represent classification rules based on which leaf nodes are assigned class labels.
<img src="decision_tree_learner_restaurant.png" alt="Drawing" style="width: 600px;"/>
#### Decision Tree Learning
Decision tree learning is the construction of a decision tree from class-labeled training data. The data is expected to be a tuple in which each record of the tuple is an attribute used for classification. The decision tree is built top-down, by choosing a variable at each step that best splits the set of items. There are different metrics for measuring the "best split". These generally measure the homogeneity of the target variable within the subsets.

#### Gini Impurity
Gini impurity of a set is the probability of a randomly chosen element to be incorrectly labeled if it was randomly labeled according to the distribution of labels in the set.

$$I_G(p) = \sum{p_i(1 - p_i)} = 1 - \sum{p_i^2}$$

We select a split which minimizes the Gini impurity in child nodes.

#### Information Gain
Information gain is based on the concept of entropy from information theory. Entropy is defined as:

$$B(p) = -\sum{p_i \log_2{p_i}}$$

Information Gain is difference between entropy of the parent and weighted sum of entropy of children. The feature used for splitting is the one which provides the most information gain.

#### Pseudocode

You can view the pseudocode by running the cell below:

In [ ]:
pseudocode("Decision Tree Learning")

### Implementation
The nodes of the tree constructed by our learning algorithm are stored using either `DecisionFork` or `DecisionLeaf` based on whether they are a parent node or a leaf node respectively.

In [ ]:
psource(DecisionFork)

`DecisionFork` holds the attribute, which is tested at that node, and a dict of branches. The branches store the child nodes, one for each of the attribute's values. Calling an object of this class as a function with input tuple as an argument returns the next node in the classification path based on the result of the attribute test.

In [ ]:
psource(DecisionLeaf)

The leaf node stores the class label in `result`. All input tuples' classification paths end on a `DecisionLeaf` whose `result` attribute decide their class.

In [ ]:
psource(DecisionTreeLearner)

The implementation of `DecisionTreeLearner` provided in [learning.py](https://github.com/aimacode/aima-python/blob/master/learning.py) uses information gain as the metric for selecting which attribute to test for splitting. The function builds the tree top-down in a recursive manner. Based on the input it makes one of the four choices:
<ol>
<li>If the input at the current step has no training data we return the mode of classes of input data received in the parent step (previous level of recursion).</li>
<li>If all values in training data belong to the same class it returns a `DecisionLeaf` whose class label is the class which all the data belongs to.</li>
<li>If the data has no attributes that can be tested we return the class with highest plurality value in the training data.</li>
<li>We choose the attribute which gives the highest amount of entropy gain and return a `DecisionFork` which splits based on this attribute. Each branch recursively calls `decision_tree_learning` to construct the sub-tree.</li>
</ol>

### Example

We will now use the Decision Tree Learner to learn a decision tree for the example from **lecture 13, slide 13**.
First, create the dataset.

In [ ]:
attrnames = ['Alt', 'Bar', 'Fri', 'Hun', 'Pat', 'Price', 'Rain', 'Res', 'Type', 'Est', 'WillWait']
examples = [
            ['T', 'F', 'F', 'T', 'Some', '$$$', 'F', 'T', 'French',  '0-10',  'T'],
            ['T', 'F', 'F', 'T', 'Full', '$',   'F', 'F', 'Thai',    '30-60', 'F'], 
            ['F', 'T', 'F', 'F', 'Some', '$',   'F', 'F', 'Burger',  '0-10',  'T'], 
            ['T', 'F', 'T', 'T', 'Full', '$',   'F', 'F', 'Thai',    '10-30', 'T'], 
            ['T', 'F', 'T', 'F', 'Full', '$$$', 'F', 'T', 'French',  '>60',   'F'], 
            ['F', 'T', 'F', 'T', 'Some', '$$',  'T', 'T', 'Italian', '0-10',  'T'], 
            ['F', 'T', 'F', 'F', 'None', '$',   'T', 'F', 'Burger',  '0-10',  'F'], 
            ['F', 'F', 'F', 'T', 'Some', '$$',  'T', 'T', 'Thai',    '0-10',  'T'], 
            ['F', 'T', 'T', 'F', 'Full', '$',   'T', 'F', 'Burger',  '>60',   'F'], 
            ['T', 'T', 'T', 'T', 'Full', '$$$', 'F', 'T', 'Italian', '10-30', 'F'], 
            ['F', 'F', 'F', 'F', 'None', '$',   'F', 'F', 'Thai',    '0-10',  'F'], 
            ['T', 'T', 'T', 'T', 'Full', '$',   'F', 'F', 'Burger',  '30-60', 'T']
           ]

lecture_dataset = DataSet(examples=examples, attrnames=attrnames, target='WillWait')
lecture_dataset

In [ ]:
DTL = DecisionTreeLearner(lecture_dataset)

Take a look at the tree.

In [ ]:
DTL

Display the tree properly.

In [ ]:
DTL.display()

<div class="alert alert-info">
    <h3>Note</h3>
    <p>The calculated decision tree could be different from the one showed in the lecture slide 19. The reason is that after spliting the samples with Patrons, the information gain for the five attributes: Hun, Price, Res, Type, and Est are equal. Therefore, the algorithm randomly choose one of the five attribute and split the samples further. We illustrate this by calculating the information gain values in the following</p> 
</div>

<img src="decision_tree_restaurant_result.png" alt="Drawing" style="width: 500px;"/>

We adapt the functions from learning.py first.

In [ ]:
def split_by(attr, examples, values):
        """Return a list of (val, examples) pairs for each val of attr."""
        return [(v, [e for e in examples if e[attr] == v])
                for v in values[attr]]

def count(attr, val, examples):
    """Count the number of examples that have example[attr] = val."""
    return sum(e[attr] == val for e in examples)

def information_gain(attr, examples, values, target):
    """Return the expected reduction in entropy from splitting by attr."""
    def I(examples):
        return information_content([count(target, v, examples)
                                    for v in values[target]])
    N = len(examples)
    remainder = sum((len(examples_i)/N) * I(examples_i)
                    for (v, examples_i) in split_by(attr, examples, values))
    return I(examples) - remainder

We get three child datasets after spliting the samples with attribute Patrons. All the samples of the child dataset for Pat="None" and for Pat="Some" have the same value for the target. So we only need to split the tree further for Pat="Full". We define the child dataset first:

In [ ]:
attrnames = ['Alt', 'Bar', 'Fri', 'Hun', 'Price', 'Rain', 'Res', 'Type', 'Est', 'WillWait']
examples = [
            ['T', 'F', 'F', 'T', '$',   'F', 'F', 'Thai',    '30-60', 'F'], 
            ['T', 'F', 'T', 'T', '$',   'F', 'F', 'Thai',    '10-30', 'T'], 
            ['T', 'F', 'T', 'F', '$$$', 'F', 'T', 'French',  '>60',   'F'],
            ['F', 'T', 'T', 'F', '$',   'T', 'F', 'Burger',  '>60',   'F'], 
            ['T', 'T', 'T', 'T', '$$$', 'F', 'T', 'Italian', '10-30', 'F'],
            ['T', 'T', 'T', 'T', '$',   'F', 'F', 'Burger',  '30-60', 'T']
           ]
child_dataset = DataSet(examples=examples, attrnames=attrnames, target='WillWait')
child_dataset

In [ ]:
for attr in child_dataset.inputs:
    info_gain = information_gain(attr, child_dataset.examples, 
                                 child_dataset.values, child_dataset.target)
    print("Information gain of attribute {} is {}".format(child_dataset.attrnames[attr], info_gain))

We can see that the information gain of Hun, Price, Res, Type, and Est are all 0.2516. So it does not matter along which of the five attributes to split further. 

Now we use the learned decision tree to classify a sample with 
    'Alt'= 'F', 'Bar'='F', 'Fri'='F', 'Hun='F', 'Pat'='None', 'Price'='$', 'Rain'='F', 'Res'='F', 'Type'='Thai', 'Est'='0-10'.

In [ ]:
print(DTL(['F', 'F', 'F', 'F','None', '$', 'F', 'F', 'Thai', '0-10']))

As expected, the Decision Tree learner classifies the sample as "F".